In [2]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [3]:
aqi_agg = pd.DataFrame(columns=['Date', 'Parameters', 'Measurement','AQS_ID'])

for j in range(1,72):
    file_name = f'airnow_data_{j}.csv'
    aqi_data = pd.read_csv(file_name,header=None,
                           names=['Datetime','Parameters','Measurement','AQS_ID'],
                           usecols=[2,3,4,11])
    aqi_data['Datetime'] = pd.to_datetime(aqi_data['Datetime'])
    aqi_data['Date'] = aqi_data['Datetime'].dt.date
    aqi_data = aqi_data.drop('Datetime',axis=1)
    column_move = aqi_data.pop('Date')
    aqi_data.insert(0,'Date',column_move)
    aqi_agg = pd.concat([aqi_agg, aqi_data], ignore_index=True)

AQS_to_Division = pd.read_csv('AQS_to_Climate_Division.csv')
aqi_agg = aqi_agg.merge(AQS_to_Division, on='AQS_ID', how='left')

    # AQS_Missing_Division = set()
    # missing = aqi_data[aqi_data['Division'].isna()]['AQS_ID'].unique()
    # print(j)
    # print(missing)
    # if len(missing)>0:
        # AQS_Missing_Division.add(missing[0])
    # print(AQS_Missing_Division)

C:\Users\seanf\AppData\Local\Temp\ipykernel_18620\3342498814.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  aqi_agg = pd.concat([aqi_agg, aqi_data], ignore_index=True)


In [ ]:
aqi_agg = aqi_agg.pivot_table(index=['Date', 'Division'], 
                                columns='Parameters', 
                                values='Measurement',
                                aggfunc='mean')
aqi_agg.columns.name=None
aqi_agg = aqi_agg.round(1)
aqi_agg = aqi_agg.reset_index()
aqi_agg = aqi_agg.sort_values(by=['Division','Date']).reset_index(drop=True)

aqi_agg.to_csv('airnow_api_final.csv', index=False, encoding='utf-8')